In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import math
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,ID,timestamp,Stock_ID,Volume,Three_Day_Moving_Average,Five_Day_Moving_Average,Ten_Day_Moving_Average,Twenty_Day_Moving_Average,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement,Outcome
0,1_1,1,1,-0.33308,-0.99681,-0.98736,-0.94728,-0.91977,-0.21129,-0.38778,-0.39903,-0.14321,1
1,3_1,1,3,-0.67096,-2.08895,-2.08251,-2.02183,-1.93779,-0.46476,0.97308,-0.31078,0.80286,0
2,4_1,1,4,-0.22040,1.07839,1.09143,1.18737,NaN,-0.51638,-0.52368,-0.24140,-0.31910,0
3,6_1,1,6,-0.23118,-0.55106,-0.53976,-0.47569,-0.42449,-0.60816,-0.69311,-0.40673,-0.45965,0
4,10_1,1,10,0.15076,-1.40937,-1.39805,-1.36756,-1.31365,-0.33206,-0.43690,-0.33347,0.29217,0


In [12]:
col_name = train.columns[1:]
col_name

Index([u'timestamp', u'Stock_ID', u'Volume', u'Three_Day_Moving_Average',
       u'Five_Day_Moving_Average', u'Ten_Day_Moving_Average',
       u'Twenty_Day_Moving_Average', u'True_Range', u'Average_True_Range',
       u'Positive_Directional_Movement', u'Negative_Directional_Movement',
       u'Outcome', u'mean_moving'],
      dtype='object')

In [6]:
train['mean_moving'] = (train['Three_Day_Moving_Average'] + train['Five_Day_Moving_Average'] + train['Ten_Day_Moving_Average'] + train['Twenty_Day_Moving_Average'])/4

In [60]:
train1 = train[['Stock_ID','Volume','True_Range','Average_True_Range','Positive_Directional_Movement','Negative_Directional_Movement','mean_moving']]

In [61]:
train1.head()

,Stock_ID,Volume,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement,mean_moving
0,1,-0.33308,-0.21129,-0.38778,-0.39903,-0.14321,-0.962805
1,3,-0.67096,-0.46476,0.97308,-0.31078,0.80286,-2.032770
2,4,-0.22040,-0.51638,-0.52368,-0.24140,-0.31910,NaN
3,6,-0.23118,-0.60816,-0.69311,-0.40673,-0.45965,-0.497750
4,10,0.15076,-0.33206,-0.43690,-0.33347,0.29217,-1.372157


In [62]:
train1 = train1.fillna(train1.median())

In [67]:
import xgboost as xgb
xgb1 = xgb.XGBClassifier(
        learning_rate =0.1,
        n_estimators=50,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb1.fit(train1,train['Outcome'])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=50, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

In [71]:
test1 = test
test1['mean_moving'] = (test1['Three_Day_Moving_Average'] + test1['Five_Day_Moving_Average'] + test1['Ten_Day_Moving_Average'] + test1['Twenty_Day_Moving_Average'])/4

In [72]:
test1 = test1[['Stock_ID','Volume','True_Range','Average_True_Range','Positive_Directional_Movement','Negative_Directional_Movement','mean_moving']]

In [73]:
test1.head()

,Stock_ID,Volume,True_Range,Average_True_Range,Positive_Directional_Movement,Negative_Directional_Movement,mean_moving
0,1,0.43959,0.11595,0.17058,-0.39903,-0.33365,-1.039813
1,2,-0.38494,-0.47816,-0.48696,-0.28545,-0.30166,-0.632525
2,3,1.38417,0.27538,0.86787,-0.31078,-0.43770,-0.345160
3,4,-0.22040,-0.51638,0.04259,-0.24140,-0.31910,-1.641583
4,6,-0.21500,0.10491,-0.63238,-0.40673,-0.45965,0.140720


In [74]:
test1 = test1.fillna(test1.median())

In [76]:
temp = pd.DataFrame()
temp['ID'] = test['ID']
temp['Outcome'] = xgb1.predict(test1)
temp.to_csv('xgb2.csv', header=True, index = False)

In [75]:
xgb1.predict(test1).mean()

0.11093127734290703

In [38]:
pd.crosstab(index=train1['timestamp'],columns="count")

col_0,count
timestamp,
1,1730
2,1731
3,1733
4,1735
5,1733
8,1735
9,1735
10,1735
11,1736


In [39]:
pd.crosstab(index=test1['timestamp'],columns="count")

col_0,count
timestamp,
554,1954
561,1
568,1957
575,1959
582,1961
589,1966
596,1967
603,1972
610,1971


In [47]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
ext=ExtraTreesClassifier(n_estimators=50, oob_score=True, bootstrap=True)
ext.fit(train1,train['Outcome'])
#Run predictions on the Test set


ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [46]:
train1.isnull().any()

timestamp                        False
Stock_ID                         False
Volume                           False
True_Range                       False
Average_True_Range               False
Positive_Directional_Movement    False
Negative_Directional_Movement    False
mean_moving                      False
dtype: bool

In [51]:
y_pred=ext.predict(test1)

In [52]:
y_pred.mean()

0.016440076118729524

In [83]:
#Always call fit on the estimator before invoking this method.
def importances(estimator, col_array, title):

# Calculate the feature ranking - Top 10
    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1] #It returns the indices that would sort an array.

    print "%s Top 20 Important Features\n" %title

    for f in range(7):
        print("%d. %s   (%f)" % (f + 1, col_array.columns[indices[f]], importances[indices[f]]))
    
#Mean Feature Importance
    print "\nMean Feature Importance %.6f" %np.mean(importances)
    
#Plot the feature importances of the forest
    indices=indices[:10]
    plt.figure()
    plt.title(title+" Top 10 Feature importances")
    plt.bar(range(10), importances[indices],
            color="gr", align="center")
    plt.xticks(range(10), col_array.columns[indices], fontsize=14, rotation=90)
    plt.xlim([-1, 10])
    plt.show()

In [84]:
importances(xgb1, train1, "Cover Type (Initial RF)")

Cover Type (Initial RF) Top 20 Important Features

1. Negative_Directional_Movement   (0.179470)
2. Volume   (0.174834)
3. Positive_Directional_Movement   (0.166225)
4. mean_moving   (0.164901)
5. True_Range   (0.140397)
6. Average_True_Range   (0.110596)
7. Stock_ID   (0.063576)

Mean Feature Importance 0.142857


NameError: global name 'plt' is not defined